In [ ]:

https://opendatabim.io/index.php/checking-the-quality-of-revit-and-ifc-projects/

# Solution logic
With the help of a table describing the parameters in the category to be checked and with the help of Pipelines from OpenDataBIM – **we check the projects in Revit and IFC formats**. We automatically generate a PDF document, which contains the basic information about the parameters and categories to be checked.

> In the construction industry, having accurate and high-quality building information models (BIM) is essential to the successful completion of a project. 

![](https://opendatabim.io/wp-content/uploads/2023/02/Solution-Check-Data-Revit-IFC.gif)


#### Read more about OpenDataBIM verification solution on the website:
#### https://opendatabim.io/index.php/checking-the-quality-of-revit-and-ifc-projects/

In [ ]:
### installing the necessary Python libraries via a file requirements.txt
### uncomment the line (remove the grid sign from the next line)
# pip install -r requirements.txt

### or install each library individually
# pip install fpdf
# pip install # pip install missingno
# pip install pandas

In [2]:
# Required installation of a library for creating PDF documents
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40725 sha256=2c4d3b9f9c0a3192833bf874e159fcf28fb328b40b378b54f1d0ea8a4d93ccd9
  Stored in directory: /root/.cache/pip/wheels/d7/ca/c8/86467e7957bbbcbdf4cf4870fc7dc95e9a16404b2e3c3a98c3
Successfully built fpdf
Note: you may need to restart the kernel to use updated packages.


In [6]:
#####################################################################################
# OpenDataBIM
# URI: https://opendatabim.io/
# If you have a feature enhancement request, request a price by info@opendatabim.io

# Pipeline:  Batch reporting for Revit and IFC projects
# Description: Creating PDF documents by criteria from an Excel spreadsheet for Revit and IFC files
# Input:  1. Tables with criteria, 2. project files in tabular form
# Output: PDF report for each file 

# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#####################################################################################

import json, os, re, subprocess, warnings, pandas as pd
from datetime import date
from fpdf import FPDF
import matplotlib.pyplot as plt
import missingno as msno
import logging
logging.getLogger('matplotlib.font_manager').disabled = True
warnings.simplefilter(action='ignore', category=FutureWarning)


############################   Parameters    ############################

# Folders where conversion files are located
path = r'/kaggle/input/check-the-projects-in-revit-and-ifc-formats/'

# Folders where Table with Parameters are located
pathfold = r'/kaggle/input/table-of-parameters-pdf-content-sources/'
namemap = pathfold + 'ODB Table_of_Parameters_Revit _IFC_Check.xlsx'

#########################################################################


# Prerequisites 
<span style="font-size: 14pt;">Before you dive into running the application, you need to get the <span style="color: #333399;"><strong>data from the project as a CSV or XLSX</strong></span> <code>Excel</code>&nbsp;table. To export Revit or IFC format into tables, you can use any solution you work with:<span style="font-size: 14pt;"><span style="color: #333399;"><strong>👨‍💻 Manual extraction of tables</strong><span style="color: #000000;">:</span> <code>Revit</code> & <code>Dynamo</code>, <code>Revit</code> & <code>Schedule</code>, <code>Revit</code> & <code>ODBC</code>, <code>pyRevit</code>, <code>Forge</code>, <code>SimpleBIM</code>, <code>Desite</code>, <code>IfcOpenShell</code>, <code>IFCjs</code> <span style="color: #999999;">and others</span></span><span style="font-size: 14pt;"><span style="color: #333399;"><strong> ⚙️ Automatic and batch table </strong> <span style="color: #000000;">retrieval:</span></span> using <a href="https://opendatabim.io/index.php/product/opendatabim-converter-kit-lite-version/"><code>noBIM Lite</code></a> converters <span style="color: #999999; font-size: 12pt;">(<span class="css3_grid_vertical_align_table"><span class="css3_grid_vertical_align">Revit 2018, Revit 2019, Revit 2020, Revit 2021,&nbsp;</span></span></span><span class="css3_grid_vertical_align_table"><span class="css3_grid_vertical_align"><span style="color: #999999; font-size: 12pt;">IFC2X3, IFC4X1, IFC4X, IFC4 - IFC4.3),</span> <a href="https://opendatabim.io/index.php/product/opendatabim-converter"><code>noBIM Full</code></a> converters <span style="color: #999999; font-size: 12pt;">(</span></span></span><span style="color: #999999; font-size: 12pt;"><span class="css3_grid_vertical_align_table"><span class="css3_grid_vertical_align">Revit 2015, Revit 2016, Revit 2017, Revit 2018, Revit 2019, Revit 2020, Revit 2021, Revit 2022, Revit 2023</span></span><span class="css3_grid_vertical_align_table"><span class="css3_grid_vertical_align">,&nbsp;</span></span><span class="css3_grid_vertical_align_table"><span class="css3_grid_vertical_align">IFC2X3, IFC4X1, IFC4X, IFC4 - IFC4.3)</span></span></span></span> 
    
![](https://opendatabim.io/wp-content/uploads/2023/02/github.com-OpenDataBIM-5.gif) 

## The code below only needs to be used if you have OpenDataBIM converter: **noBIM Community, noBIM Lite, noBIM Full**<br>
### noBIM Lite: https://opendatabim.io/index.php/product/opendatabim-converter-kit-lite-version/ 
### noBIM Full: https://opendatabim.io/index.php/product/opendatabim-converter/

In [7]:
#  The code below only needs to be used if you have OpenDataBIM converter: noBIM Community, noBIM Lite, noBIM Full

"""
import os, subprocess
import time

############################   Parameters    ############################

# path to noBIM converter
path_conv = r'C:\OpenDataBIM\ODBLite\noBIM_Lite_v1_23-v2jfja\\'

#########################################################################

# output path
outpath = path
# Folders where the converter and conversion files are located
conv_IfcToCsv = path_conv + 'IfcToCsv.exe'
conv_IfcColladaExporter = path_conv + 'IfcColladaExporter.exe'
conv_RvtToCsv = path_conv + 'RvtToCsv.exe'
conv_RvtColladaExporter = path_conv + 'RvtColladaExporter.exe'

def convert(path, path_conv, outpath):
    try: os.mkdir(outpath)
    except: pass

    # Conversion process from RVT and IFC in DAE
    for file in os.listdir(path):
        filepath = path + file
        fileoutpath = outpath + file[:-3]
        
        # Conversion process from IFC in DAE and CSV
        if file.endswith('.ifc'):
            subprocess.Popen([conv_IfcToCsv, filepath, fileoutpath+'csv'], cwd = path_conv)
            subprocess.Popen([conv_IfcColladaExporter, filepath, fileoutpath+'dae'], cwd = path_conv)
            while not os.path.exists(fileoutpath+'dae') and not os.path.exists(fileoutpath+'csv'):
                time.sleep(0.3)
            print("Conversion Done: " + file[:-3]+'csv' + ', ' + file[:-3]+'dae' )
            
        # Conversion process from Revit in DAE and CSV
        if file.endswith('.rvt'):
            subprocess.Popen([conv_RvtColladaExporter, filepath, fileoutpath+'dae'], cwd = path_conv)
            while not os.path.exists(fileoutpath+'dae'):
                time.sleep(0.3)
            print("Conversion Done: " + file[:-3]+'dae' )
            while not os.path.getsize(fileoutpath+'dae') > os.path.getsize(filepath)*0.7:
                time.sleep(0.3)
            time.sleep(1)
            subprocess.Popen([conv_RvtToCsv, filepath, fileoutpath+'dae', fileoutpath+'csv'], cwd = path_conv)
            while not os.path.getsize(fileoutpath+'dae') > 1000:
                time.sleep(0.3)
            print("Conversion Done: " + file[:-3]+'csv')         
        time.sleep(7)
    return

# Run a function to search for any RVT and IFC files in a folder and then convert them
convert(path, path_conv, outpath)
"""

'\nimport os, subprocess\nimport time\n\n############################   Parameters    ############################\n\n# path to noBIM converter\npath_conv = r\'C:\\OpenDataBIM\\ODBLite\noBIM_Lite_v1_23-v2jfja\\\'\n\n#########################################################################\n\n# output path\noutpath = path\n# Folders where the converter and conversion files are located\nconv_IfcToCsv = path_conv + \'IfcToCsv.exe\'\nconv_IfcColladaExporter = path_conv + \'IfcColladaExporter.exe\'\nconv_RvtToCsv = path_conv + \'RvtToCsv.exe\'\nconv_RvtColladaExporter = path_conv + \'RvtColladaExporter.exe\'\n\ndef convert(path, path_conv, outpath):\n    try: os.mkdir(outpath)\n    except: pass\n\n    # Conversion process from RVT and IFC in DAE\n    for file in os.listdir(path):\n        filepath = path + file\n        fileoutpath = outpath + file[:-3]\n        \n        # Conversion process from IFC in DAE and CSV\n        if file.endswith(\'.ifc\'):\n            subprocess.Popen([conv_If

# Excel with checking rules
<span style="font-size: 14pt;">Any project elements in <code>Revit </code>and <code>IFC </code>format belong to some group (<code>OST_ </code>in <code>Revit</code>, <code>IfcEntity </code>in <code>IFC</code>) or type group (<code>TypeName </code>in <code>Revit</code>, <code>ObjectType </code>in <code>IFC</code>). Also, <span style="color: #333399;"><strong>each element has its own unique set of properties and parmeters</strong> </span>(which are columns in the overall project table when the <code>noBIM </code>concept is applied).</span>

<span style="font-size: 14pt;">Using the check table, <span style="color: #333399;">we specify in the first column of the table, </span><span style="color: #000000;">for the elements to be checked and tested</span>, <span style="color: #333399;">first the category</span> (<code>OST_ </code>in <code>Revit</code>, <code>IfcEntity </code>in <code>IFC</code>) or type (<code>TypeName</code>in <code>Revit</code>, <code>ObjectType </code>in <code>IFC</code>) and in the <span style="color: #333399;"><strong>second column we specify the parameters</strong> </span>that we want to check for this group of elements.</span>
![](https://opendatabim.io/wp-content/uploads/2023/02/Excel-Check-Revit-IFC-Project.gif)

<span style="font-size: 14pt;">For each parameter from the check table in the project will be checked:</span>
<span style="font-size: 14pt;"><span style="color: #333399;">
<ul style="list-style-type: square;">
 	<li><span style="font-size: 14pt;"><span style="color: #333399;"><strong>presence of the parameter</strong> </span>for the group of elements</span></li>
 	<li><span style="font-size: 14pt;"><span style="color: #333399;"><strong>percentage of the content of the values</strong></span> in the parameter</span></li>
 	<li><span style="font-size: 14pt;">the <span style="color: #333399;"><strong>unique values</strong></span> of this parameter</span></li>
</ul>

In [19]:
# Where files and photos of project data are created
pathr = pathfold + 'PDF_Content_Sources/'
pathro = path
#pathro = path + r'Geometry\\'

# Variables used in the code
# For Revit - Category, for IFC - IfcEntity
param_group = 'Category'

# Which files will be checked
csv_files = []
for file in os.listdir(path):
    if file.endswith('.csv'):
        csv_files.append(file)

csv_files

['rac_advanced_sample_project_rvt.csv',
 'rac_basic_sample_project_rvt.csv',
 'rme_basic_sample_project_rvt.csv',
 'Technical_school-current_m_rvt.csv']

In [20]:
# Classes used to format a PDF document
class PDF(FPDF):
    def header(self):
        # Arial bold 15
        self.set_font('Arial', 'B', 12)
        # Move to the right
        # Title
        self.set_text_color(56, 81, 153)
        self.cell(1, 0, '', 50, 100, 'L')
        self.cell(1, 6, 'BIM PARAMETER CHECK', 50, 1, 'L',  link = '')
        self.cell(1, 6, projectf[:-4], 50, 200, 'L')
        # Logo
        self.image(pathr + 'logo_odb.png', 25, 3, 175, )
        # Line break
        self.ln(6)
        self.set_text_color(0, 0, 0)

    # Page footer
    def footer(self):
        #if self.page_no() != 1:
        # Position at 1.5 cm from bottom
        self.set_y(-20)
        # Arial italic 8
        self.set_font('Arial', '', 8)
        # Page number
        self.image(pathr + 'footer_odb.png', x = 24, y = None, w = 170, h = 0, type = '', link = 'https://opendatabim.io/')
        self.set_text_color(56, 81, 153)
        self.cell(170, 4, d2 + '  ' + 'Page  ' + str(self.page_no()) + '/{nb}', 0, 0, 'R')

# Run the code
 <span>With these steps, you should be <span style="color: #333399;"><strong>able to run Python code on a new computer without any issues</strong></span>.</span></br>
 
After the script runs, check the finished reports on the right side in the "Output" section</br>
![](https://opendatabim.io/wp-content/uploads/2023/02/github.com-OpenDataBIM-6.gif)

In [23]:
# Read parameter table Excel
df_param =  pd.read_excel(namemap, header = 1)   
listpar = list(zip(df_param['REVIT category'], df_param['Parameters to check']))

today = date.today()
d2 = today.strftime("%d.%m.%Y")

# Go through all the files in the folder
for file in csv_files:  
    
    # Creating a dictionary with all the criteria
    dictl = {} 
    for el in listpar:
        dictl.setdefault(el[0], []).append(el[1])
    
    pdf = PDF()
    pdf.alias_nb_pages()

    # Page indentation
    pdf.set_left_margin(25)
    pdf.set_right_margin(15)
    
    # Project name in the document header
    projectf = file 
    strnr = 1 
    countab = 1
    
    # Reading a project file
    df = pd.read_csv(path + file)
    pathf = '../' + file[:-4] + '_IMG'
    
    # Creating folders
    try: os.mkdir(pathro)
    except: pass
    try: os.mkdir(pathf)
    except: pass   

    file_dae = path + file.replace('.csv', '.dae')
    try: ImgCapture.load(file_dae)
    except: print(file_dae)
    
    # Parameter
    ent_value = df[param_group].unique()
    dict_wrong = {}
    klnummer = 1
    
    # All categories of the project
    ent_value = [item for item in ent_value if not(pd.isnull(item)) == True]
    
    # Going through all categories of the project       
    for elkl in ent_value:
        if elkl in dictl.keys():    
            
            # Creating a data frame for a specific group
            klasse_dfmatch, klasse_dfmatchw = [], []
            dfparam = df.loc[df[param_group] == elkl]
            dfparam = dfparam.dropna(axis=1, how='all')
            klasse_dfmatch = dfparam.columns
            df_klasse = df.loc[:, klasse_dfmatch]
            df_klasse = df_klasse.loc[df_klasse[param_group] == elkl]
            df_klasse = df_klasse.dropna(how='all', axis=0)

            # Receiving data that does not meet the criteria
            n_inproject = []    
            odb_param = []
            if elkl in dictl.keys():
                parameter_tab = dictl[elkl]
                param_len = len(parameter_tab)
                for el in df_klasse.columns[:45].to_list():
                    if el in parameter_tab:
                        odb_param.append(el)
                    else:
                        n_inproject.append(el)
            
            # Obtaining the table of parameters distributed by the number of values
            nval = [df_klasse[col_name].count() for col_name in df_klasse.columns]
            dict_nv = {}
            dict_nv = dict(zip(df_klasse.columns, nval))
            dict_nv = sorted(dict_nv.items(), key=lambda x: x[1], reverse=True)
            
            # Parameters that are not in the table to check
            dict_nvn = []
            for el in dict_nv:
                dict_nvn.append(el[0])
            for el in odb_param:
                dict_nvn.remove(el)
            
            for el in odb_param:
                dict_nvn.insert(0, el)
            df_klasse = df_klasse[dict_nvn]
            
            # Parameters found in XLSX table to check
            parameter_s23 = parameter_tab 
            for el23 in odb_param:
                parameter_s23.remove(el23)
            
            # Defining colors for parameters that are found and parameters that were not found
            colorsd = []
            for el in odb_param:
                colorsd.append('#43CD62')
            for el in range(len(dict_nvn)):
                colorsd.append('grey')
            # Creating a picture PNG for all parameters
            namefig = msno.bar(df_klasse[df_klasse.columns[:45]], figsize=(25,4), fontsize=20, color=colorsd)
            fig_copy = namefig.get_figure()
            elklclean = re.sub('[^A-Za-z0-9]+', '', elkl)
            
            # Save the picture in the folder
            fig_copy.savefig(pathf + '\\' + elklclean + '_attributesklasse.png', bbox_inches = 'tight')
            namefig.clear()
            plt.clf()

            df_klasse = df_klasse.set_index('ID')
            df_klasse.index = df_klasse.index.astype(str)
            nodes = list(df_klasse.index)
            
            # Forming geometry for elements in the group at the top of the document
            level_img = pathro + file[:30] + elklclean + '_geometry_image.png'
            #img = ImgCapture.shoot(level_img, nodes)

            
            #bookm[klne + ' ' + param_group + ': '+ elkl]=pdf.page_no()
            
            ################################################################
            ###################### PDF document ############################
            ################################################################
            
            # Creating a new page of a PDF document
            pdf.add_page()
            klnummer += 1
            
            ################################################################
            # If the group photo is generated
            try: pdf.image(level_img, x = 55, y = 32, w = 65, type = '')
            except: pass
            pdf.set_font('Arial', 'B', 10)
            pdf.set_text_color(0,102,204)
            pdf.set_font('','U')
            pdf.set_font('Arial', '', 10)
            pdf.set_text_color(0, 0, 0)
                

            ################################################################

            # Group name at the top of the document
            klne = str(strnr) + '.' + str(klnummer)    
            pdf.set_font('Arial', 'B', 12)
            pdf.set_fill_color(241, 241, 242)
            pdf.cell(len(param_group) + 24, 7, ' ' + klne + ' ' + param_group + ': ', 0, 0, 'L', fill = True)
            pdf.cell(3, 7, ' ', 0, 0, 'L')
            pdf.set_fill_color(210, 255, 224)
            pdf.cell(len(elkl) + 20, 7, ' ' + elkl, 0, 1, 'C', fill = True)
            pdf.ln(57)
            
             ################################################################
                
            pdf.set_font('Arial', '', 10)
            # Check the number of parameters found, if more than 50% 
            if len(odb_param)/param_len > 0.5:
                pdf.image(pathr + 'v.png', x = 125, y = 57, w = 75)
            else:
                pdf.image(pathr + 'w.png', x = 130, y = 58, w = 70)
            
            ################################################################
            
            # Forming a table with the statistics of the parameters checked and as a whole
            
            pdf.set_font('Arial', '', 8)
            pdf.cell(32, 7, 'Checking parameters by criteria from an Excel table:', 0, 1, 'L')
           
            pdf.set_font('Arial', 'B', 10)
            pdf.set_fill_color(225, 225, 225)
            pdf.cell(32, 7, 'Elements', 0, 0, 'C', fill = True)
            pdf.cell(3, 7, '', 0, 0, 'c') 
            pdf.cell(32, 7,  'Parameters', 0, 0, 'C', fill = True) 
            pdf.cell(3, 7, '', 0, 0, 'c') 
            pdf.cell(32, 7, 'Checked', 0, 0, 'C', fill = True) 
            pdf.cell(3, 7, '', 0, 0, 'c') 
            pdf.set_fill_color(165, 250, 184)
            pdf.cell(32, 7,  'Project has', 0, 0, 'C', fill = True) 
            if len(parameter_s23) == 0:
                pdf.set_fill_color(165, 255, 200)
            else:
                pdf.set_fill_color(250, 180, 165)
            strparam = str(parameter_s23)
            pdf.cell(3, 7, '', 0, 0, 'c') 
            strparam = strparam.replace("[", "")
            strparam = strparam.replace("]", "")
            pdf.cell(32, 7, 'Missing', 0, 0, 'C', fill = True) 
            pdf.set_font('Arial', '', 11)
            pdf.cell(20, 8, '', 2, 1, 'L')

            ##############
            
            pdf.set_font('Arial', '', 10)
            pdf.set_fill_color(241, 241, 242)
            pdf.cell(32, 7, str(len(df_klasse)), 0, 0, 'C', fill = True)
            pdf.cell(3, 7, '', 0, 0, 'c') 
            pdf.cell(32, 7, str(len(df_klasse.columns)), 0, 0, 'C', fill = True) 
            pdf.cell(3, 7, '', 0, 0, 'c') 
            pdf.cell(32, 7, str(param_len), 0, 0, 'C', fill = True) 
            pdf.cell(3, 7, '', 0, 0, 'c') 
            pdf.set_fill_color(210, 255, 224)
            pdf.cell(32, 7,  str(len(odb_param)), 0, 0, 'C', fill = True) 
            if len(parameter_s23) == 0:
                pdf.set_fill_color(165, 255, 200)
            else:
                pdf.set_fill_color(255, 218, 210)
            pdf.cell(3, 7, '', 0, 0, 'c') 
            pdf.cell(32, 7, str(len(parameter_s23)), 0, 0, 'C', fill = True) 
            pdf.set_font('Arial', '', 11)
            pdf.cell(20, 10, '', 2, 1, 'L')
            
            ################################################################
            
            # Parameters not found in the project 
            if len(parameter_s23) > 0:
                pdf.cell(150, 9, 'Parameters not found in the project or completely empty values:', 0, 1, 'L')
                pdf.set_font('Arial', '', 10)
                for el in parameter_s23:
                    pdf.set_fill_color(255, 218, 210)
                    size = len(el)+20
                    pdf.cell(size, 6, el, 0, 0, 'C', fill = True) 
                    pdf.cell(3, 7, '', 0, 0, 'C', ) 
            else:
                pdf.cell(150, 12, '', 0, 1, 'L')

            ################################################################
    
            # Forming a table with statistics on the volume of groups
            pdf.ln(4)
            pdf.set_font('Arial', '', 12)    
            try:
                df_klasse['Volume'] = df_klasse['Volume'].astype(str).str.extract(r'([-+]?\d*\.?\d+)').astype('float')
                total_vol = int(df_klasse['Volume'].sum())
            except:
                total_vol = '0'
                pass
            try:
                df_klasse['Area'] = df_klasse['Area'].astype(str).str.extract(r'([-+]?\d*\.?\d+)').astype('float')
                total_area = int(df_klasse['Area'].sum()) 
            except:
                total_area = '0'
                pass
            try:
                df_klasse['Length'] = df_klasse['Length'].astype(str).str.extract(r'([-+]?\d*\.?\d+)').astype('float')
                total_len = int(df_klasse['Length'].sum()) 
            except:
                total_len = '0'
                pass
            pdf.cell(20, 3, '', 2, 1, 'L')
            pdf.set_font('Arial', '', 8)
            pdf.cell(150, 7, 'Volumetric parameters for all elements in the groups:', 0, 1, 'С')
            
            ########
            
            pdf.set_font('Arial', 'B', 10)
            pdf.set_fill_color(225, 225, 225)
            pdf.cell(55, 7, 'Sum of volumes', 0, 0, 'C', fill = True)
            pdf.cell(3, 7, '', 0, 0, 'c') 
            pdf.cell(55, 7,  'Sum of areas', 0, 0, 'C', fill = True) 
            pdf.cell(3, 7, '', 0, 0, 'c') 
            pdf.cell(55, 7, 'Sum of lengths', 0, 0, 'C', fill = True) 
            pdf.cell(3, 7, '', 0, 1, 'c') 
            
            ########
            
            pdf.cell(20, 2, '', 2, 1, 'L')
            pdf.set_font('Arial', '', 10)
            pdf.set_fill_color(241, 241, 242)
            pdf.cell(55, 7, str(total_vol) + ' m³', 0, 0, 'C', fill = True)
            pdf.cell(3, 7, '', 0, 0, 'c') 
            pdf.cell(55, 7,  str(total_area) + ' m²', 0, 0, 'C', fill = True) 
            pdf.cell(3, 7, '', 0, 0, 'c') 
            pdf.cell(55, 7, str(total_len) + ' m', 0, 0, 'C', fill = True) 
            pdf.cell(3, 7, '', 0, 1, 'c') 
            
            ################################################################

            # Inserting a picture with the parameters that were checked above
            pdf.set_font('Arial', '', 11)
            pdf.cell(20, 5, '', 2, 1, 'L')
            pdf.set_font('Arial', '', 11)
            text1 = 'The following diagram shows which parameters in the Project ' + file[:-9] + ' are included and which parameters and how often for ' + param_group + ' ' + elkl +  ' are given:'
            pdf.multi_cell(w = 160, h =6, txt = text1, border = 0)
            pdf.cell(20, 1, '', 2, 1, 'L')
            pdf.set_font('Arial', 'B', 12)
            pdf.image(pathr + 'logo_atribbuten.png', x = 30, w = 165, type = '', link = '')
            pdf.image(pathf+ '\\' + elklclean+'_attributesklasse.png', x = 25,  h = 50, w = 165, type = '', link = '')

            ################################################################
            
            # Formation of the bottom table of statistics for the found parameters
            # Calculate the number of non-null values for each column in the dataframe using the 
            non_null_counts = dfparam.count()
            # Calculate the total number of rows in the dataframe using "len()", and store the result in "total_counts"
            total_counts = len(dfparam)
            percent_filled = non_null_counts / total_counts * 100
            # Reset the index of the dataframe "filled_df"
            filled_df = pd.DataFrame(percent_filled).reset_index()
            filled_df.columns = ['column_name', 'Percentage filling']
            # Convert the "Percentage filling" column to integer type
            filled_df['Percentage filling'] = filled_df['Percentage filling'].astype(int)

            for col in dfparam.columns:
                unique_values = dfparam[col].unique()[:5]
                filled_df.loc[filled_df['column_name'] == col, 'Example values'] = ", ".join(map(str, unique_values))

            filled_df['Example values'] = filled_df['Example values'].str.replace('[|]|nan ,|nan,|nan', '')
            filled_df['Example values'] = filled_df['Example values'].str[:50]
            filled_df = filled_df.set_index('column_name')
            
            filled_df2 = filled_df.loc[odb_param]
            pdf.ln(4)

            ###############
            if len(filled_df2) > 0:
                pdf.set_font('Arial', '', 8)
                pdf.cell(150, 7, 'Statistics on the parameters found:', 0, 1, 'С')
                pdf.set_font('Arial', 'B', 10)
                pdf.set_fill_color(165, 250, 184)
                pdf.cell(55, 6, txt='  Parameter Name', ln=0, align="L", fill = True)
                pdf.cell(3, 6, '', ln=0, align="C")
                pdf.set_fill_color(225, 225, 225)
                pdf.cell(35, 6, txt='Percentage filling', ln=0, align="C", fill = True)
                pdf.cell(3, 6, '', ln=0, align="C")
                pdf.cell(70, 6, txt='  Example values', ln=0, align="L", fill = True)
                pdf.cell(3, 7, '', ln=1, align="C")

                for i in range(len(filled_df2)):
                    pdf.set_font('Arial', '', 10)
                    pdf.set_fill_color(210, 255, 224)
                    pdf.cell(55, 6, txt='  ' + str(filled_df2.index[i][:38]), ln=0, align="L", fill = True)
                    if filled_df2.iloc[i, 0] < 50:
                        pdf.set_fill_color(255, 218, 210)
                    else:
                        pdf.set_fill_color(210, 255, 224)
                    pdf.cell(3, 6, '', ln=0, align="C")
                    pdf.cell(35, 6, txt=str(filled_df2.iloc[i, 0])+'%', ln=0, align="C", fill = True)
                    pdf.set_fill_color(241, 241, 242)
                    pdf.cell(3, 6, '', ln=0, align="C")
                    pdf.cell(70, 6, txt='  ' + str(filled_df2.iloc[i, 1][:38]), ln=0, align="L", fill = True)
                    pdf.cell(3, 7, '', ln=1, align="C") 
                    
    strnr +=1
    pdf.output(file[:-4] + 'ODB_Report.pdf', 'F')
    print('Report generated: ' + file[:-4] + 'ODB_Report.pdf')
    # After the script runs, check the finished reports on the right side in the "Output" section

/kaggle/input/check-the-projects-in-revit-and-ifc-formats/rac_advanced_sample_project_rvt.dae
Report generated: rac_advanced_sample_project_rvtODB_Report.pdf
/kaggle/input/check-the-projects-in-revit-and-ifc-formats/rac_basic_sample_project_rvt.dae
Report generated: rac_basic_sample_project_rvtODB_Report.pdf


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (8,10,13,14,15,17,20,27,28,29,30,31,33,35,36,38,41,48,53,54,56,57,58,59,62,72,73,74,75,80,81,82,83,84,85,86,87,89,90,92,93,95,96,97,98,109,110,114,115,116,117,118,123,125,126,127,128,129,130,132,133,137,138,144,145,150,151,152,173,174,182,183,189,190,191,193,194,195,196,197,198,199,200,202,203,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,276,284,285,286,287,288,289,290,292,295,296,297,298,299,300,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,318,321,322,323,327,328,329,330,331,332,333,335,338,339,340,341,343,344,345,347,351,352,354,355,356,357,359,361,362,363,364,365,367,368,369,370,371,372,373,374,375,376,378,379,380,381,383,388,391,393,394,395,396,399,400,401,402,4

/kaggle/input/check-the-projects-in-revit-and-ifc-formats/rme_basic_sample_project_rvt.dae
Report generated: rme_basic_sample_project_rvtODB_Report.pdf
/kaggle/input/check-the-projects-in-revit-and-ifc-formats/Technical_school-current_m_rvt.dae


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (8,9,11,12,13,14,75,80,81,82,83,84,85,86,93,94,95,96,99,100,101,103,105,124,127,128,130,133,134,137,140,141,142,143,144,150,151,155,156,157,162,165,166,167,168,169,172,173,175,178,180,181,182,183,184,185,186,187,188,189,190) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Report generated: Technical_school-current_m_rvtODB_Report.pdf


<Figure size 1800x288 with 0 Axes>